# Capturing and reusing JSON

This workflow takes as input a STAC Item and a STAC asset key to produce a quicklook in the PNG format.

The first step of the workflow uses `curl` to get the STAC Item json and this content is parsed to get the _assets_ element.

In pratical terms it does:

In [14]:
curl -s https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_53HPA_20210723_0_L2A | jq .assets

{
  "thumbnail": {
    "title": "Thumbnail",
    "type": "image/png",
    "roles": [
      "thumbnail"
    ],
    "href": "https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/53/H/PA/2021/7/23/0/preview.jpg"
  },
  "overview": {
    "title": "True color image",
    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
    "roles": [
      "overview"
    ],
    "gsd": 10,
    "eo:bands": [
      {
        "name": "B04",
        "common_name": "red",
        "center_wavelength": 0.6645,
        "full_width_half_max": 0.038
      },
      {
        "name": "B03",
        "common_name": "green",
        "center_wavelength": 0.56,
        "full_width_half_max": 0.045
      },
      {
        "name": "B02",
        "common_name": "blue",
        "center_wavelength": 0.4966,
        "full_width_half_max": 0.098
      }
    ],
    "href": "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2021/7/S2B_53HPA_20210723_0_L2A/L2A_PVI.tif",
    "proj:shape": 

and the second step uses the CWL expression `$( inputs.asset[inputs.asset_id].href )` to get the STAC Asset href.

This two step approach is the equivalent of:

In [13]:
curl -s https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_53HPA_20210723_0_L2A | jq .assets | jq .visual.href

"https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2021/7/S2B_53HPA_20210723_0_L2A/TCI.tif"


This workflow shows how to capture JSON and then re-use it to access its elements 

## Execute the workflow

In [15]:
cwltool workflow.cwl --stac_item https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_53HPA_20210723_0_L2A --asset_id visual

INFO /opt/conda/bin/cwltool 3.1.20231114134824
INFO Resolved 'workflow.cwl' to 'file:///workspace/cwl-guide-for-eo/notebooks/03%20Capture%20JSON%20and%20reuse/workflow.cwl'
INFO [workflow ] start
INFO [workflow ] starting step step_stac
INFO [step step_stac] start
INFO [job step_stac] /tmp/cgqr9j8n$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/cgqr9j8n,target=/QLCBEX \
    --mount=type=bind,source=/tmp/ts7cd1ka,target=/tmp \
    --workdir=/QLCBEX \
    --read-only=true \
    --log-driver=none \
    --user=1001:100 \
    --rm \
    --cidfile=/tmp/bwfiz7td/20240402144246-935852.cid \
    --env=TMPDIR=/tmp \
    --env=HOME=/QLCBEX \
    docker.io/curlimages/curl:latest \
    curl \
    https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_53HPA_20210723_0_L2A > /tmp/cgqr9j8n/message
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   S

In [5]:
cat quicklook.png | display